## 텐서플로를 이용한 딥러닝 시계열 예측 (예측모델링)

### 알려두기

  이 자료는 TensorFlow의 공식 튜토리얼 “언어 이해를 위한 변환기 모델”을 기반으로 작성되었습니다. 

  원문은 TensorFlow 공식 사이트(https://www.tensorflow.org/tutorials/structured_data/time_series?hl=en) 에서 확인 가능합니다.

  변경 사항: 원문 튜토리얼을 바탕으로 MBA 수강생들에게 맞게 재구성되었습니다.

라이브러리 준비

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from IPython.display import clear_output

전처리 데이터 로드

In [ ]:
df = pd.read_csv('./Data/jena_climate_2009_2016_preprocessed.csv', index_col=0, parse_dates=True)
df.head()

데이터 분할

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}
column_indices

In [ ]:
n = len(df)
train_df = df[0:int(n * 0.7)]
val_df = df[int(n * 0.7):int(n * 0.9)]
test_df = df[int(n * 0.9):]
num_features = df.shape[1]
print('특성 수 : ', num_features)

데이터 정규화

In [7]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
df_std = (df - train_mean) / train_std
df_std.head()

정규화 데이터 분포 시각화

In [9]:
df_std = df_std.melt(var_name='Column', value_name='Normalized')

In [ ]:
px.violin(df_std, y='Normalized', x='Column', box=True, points=False, color='Column').update_layout(showlegend=False)

인덱스 및 오프셋

In [11]:
class WindowGenerator():
    def __init__(self, name, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=['T (degC)']):
        # 원본 데이터 저장
        self.name = name
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df
        
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = { name: i for i, name in enumerate(label_columns) }
        self.column_indices = { name: i for i, name in enumerate(train_df.columns) }

        # 윈도우 매개변수 계산
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',])
    
    def split_window(self, features):
        # 입력 데이터에서 input_slice에 해당하는 부분을 추출하여 inputs 변수에 저장
        inputs = features[:, self.input_slice, :]
        
        # 입력 데이터에서 labels_slice에 해당하는 부분을 추출하여 labels 변수에 저장
        labels = features[:, self.labels_slice, :]
        
        if self.label_columns is not None:
            labels = tf.stack( [labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)
        
        # inputs의 형태를 [배치 크기, 입력 너비, 특성 수]로 설정
        inputs.set_shape([None, self.input_width, None])
        
        # labels의 형태를 [배치 크기, 라벨 너비, 특성 수]로 설정
        labels.set_shape([None, self.label_width, None])

        # inputs와 labels를 반환
        return inputs, labels
        
    # (input_window, label_window) 쌍의 데이터로 만들어준다.
    def make_dataset(self, data):
        
        data = np.array(data, dtype=np.float32)
        
       
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,  # 입력 데이터
            targets=None,  # 타겟 데이터는 없음
            sequence_length=self.total_window_size,  # 시퀀스 길이 설정
            sequence_stride=1,  # 시퀀스 간의 간격 설정
            shuffle=True,  # 데이터를 섞어서 제공
            batch_size=32,  # 배치 크기 설정
        )

        # 생성된 데이터셋을 split_window 함수를 통해 입력과 라벨로 분리합니다.
        ds = ds.map(self.split_window)

        # 최종 데이터셋 반환
        return ds
    
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

윈도우 객체 생성

In [ ]:
single_step_window = WindowGenerator(name="1Step 1Label 1Shift 윈도우", input_width=1, label_width=1, shift=1)
single_step_window

In [ ]:
wide_window = WindowGenerator(name="24Step 24Label 1Shift 윈도우", input_width=24, label_width=24, shift=1)
wide_window

In [ ]:
WindowGenerator(name="테스트", input_width=3, label_width=1, shift=3)

In [ ]:
CONV_WIDTH = 3
conv_window = WindowGenerator(name="3Step 1Label 1Shift 윈도우", input_width=CONV_WIDTH, label_width=1, shift=1)
conv_window

In [ ]:
LABEL_WIDTH = 24
INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1) # 24 + (3 - 1) = 26 (kernel 사이즈로 늘어난 윈도우 사이즈를 맞춰주기 위해 +2를 해준다)
wide_conv_window = WindowGenerator(name="26Step 24Label 1Shift 윈도우", input_width=INPUT_WIDTH, label_width=LABEL_WIDTH, shift=1)
wide_conv_window

In [ ]:
wide_to_single_window = WindowGenerator(name="24Step 1Step 1Shift 윈도우", input_width=LABEL_WIDTH, label_width=1, shift=1)
wide_to_single_window

In [ ]:
OUT_STEPS = 24
multi_window = WindowGenerator(name="24Step 24Label 24Shift 윈도우", input_width=OUT_STEPS, label_width=OUT_STEPS, shift=OUT_STEPS)
multi_window

윈도우 플러그인 
* 모델이 윈도우 제너레이터를 사용할수 있도록 기능을 확장하는 믹스인 클래스

In [19]:
# 모델 옵션 설정 -> 훈련 -> 플롯 -> 평가 를 한번에 진행시키는 믹스인 클래스
class WindowPluginMixin(object):
  # 시각화
    def plot(self, window, plot_col='T (degC)'):

        inputs, labels = window.example # tf.data.Dataset 객체는 inputs와 labels를 튜플로 리턴한다.
        max_n = min(3, len(inputs)) # 최대 서브차트 수를 결정한다.(최대 3개 까지)
        plot_col_index = window.column_indices[plot_col]

        fig = make_subplots(rows=1, cols=max_n, shared_xaxes=True, subplot_titles=[f'{i+1}번째 입력' for i in range(max_n)]) # 캔버스 준비

        for n in range(max_n): # 차트 수만큼 반복할 것이다.
            showlegend = (n == 0)  # 첫번째 플롯만 범례를 적용
            # 1. 입력 타임스텝을 라인플롯으로 그린다.
            fig.add_trace(
                go.Scatter(x=window.input_indices, y=inputs[n, :, plot_col_index], mode='lines+markers', name='Inputs', showlegend=showlegend, marker=dict(color='blue')),
                row=1, col=1+n
            )
            
            if window.label_columns:
                label_col_index = window.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue
            
            # 2. 검증 또는 테스트를 위한 레이블 타임스텝(훈련시에는 미래시점)을 마커로 찍어 준다.
            fig.add_trace(
                go.Scatter(x=window.label_indices, y=labels[n, :, label_col_index], mode='markers', name='Labels', marker=dict(color='#2ca02c', size=8), showlegend=showlegend),
                row=1, col=n+1,
            )
            # 예측 실행
            predictions = self(inputs)
                
            # 3. 예측값을 2번과 비교하기 위해 색상을 바꿔 함께 마커로 찍어 준다.
            fig.add_trace(
                go.Scatter(x=window.label_indices, y=predictions[n, :, label_col_index], mode='markers', name='Predictions', marker=dict(symbol='x', color='#ff7f0e', size=8), showlegend=showlegend),
                row=1, col=n+1,
            )
        # 가로 배치할것이기 때문에 높이는 400정도만 주고, 너비는 다 쓴다.
        fig.update_layout(height=400, title_text=f"[{self.name}] [{window.name}] 시계열 차트", showlegend=True, legend=dict(orientation="h", x=0.5, xanchor='center', y=-0.2), xaxis_title="Time [h]", yaxis_title=f'{plot_col} [normed]')  # 범례의 위치를 하단 중앙으로 설정
        fig.show()
        
  # 모델 훈련
    def execute(self, window, training=True, patience=2, MAX_EPOCHS=20):
        self.compile(loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanAbsoluteError()]) # 손실함수와 평가지표 설정
        if training :
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min') # 2회이상 손실감소가 없으면 조기종료
            self.history = self.fit(window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping]) # 훈련시작
            clear_output() # 출력 정리
        
        self.plot(window)
    
        _, val_mae = self.evaluate(window.val) # 검증 데이터 평가 (return : [loss, mae])
        _, test_mae = self.evaluate(window.test, verbose=0) # 테스트 데이터 평가 (return : [loss, mae])
        
        return pd.DataFrame({"검증 MAE": [val_mae], "테스트 MAE": [test_mae]}, index=[self.name])
        

___
### 싱글스텝 예측 모델
___

베이스라인 모델

In [20]:
class Baseline(tf.keras.Model, WindowPluginMixin):
  def __init__(self, label_index=None, name="Baseline"):
    super().__init__()
    self.label_index = label_index
    self.name = name

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index] # 레이블 값을 리턴
    return result[:, :, tf.newaxis]

In [ ]:
baseline = Baseline()
baseline_single_eval = baseline.execute(single_step_window)
baseline_single_eval

FC 모델

In [22]:
class FullyConnectedModel(tf.keras.Model, WindowPluginMixin):
    def __init__(self, units=[64, 64], name='FC-Model'):
        super().__init__()
        # 모델링
        self.smodel = tf.keras.Sequential()
        for unit in units :
            self.smodel.add(tf.keras.layers.Dense(unit,  activation='relu'))
        self.smodel.add(tf.keras.layers.Dense(units=1))
        self.name = name

    def call(self, inputs):
        return self.smodel(inputs)

In [ ]:
fc = FullyConnectedModel()
fc_eval = fc.execute(wide_to_single_window)
fc_eval

컨볼루셔널 FC 모델

In [24]:
class ConvFullyConnectedModel(tf.keras.Model, WindowPluginMixin):
    def __init__(self, units=[64, 64], name='Conv-FC-Model'):
        super().__init__()
        # 모델링
        self.smodel = tf.keras.Sequential()
         # Flatten 층 추가 : 입력(time, features)을 (time x features)로 펼쳐준다. 2D -> 1D
        self.smodel.add(tf.keras.layers.Flatten())
        
        # 기존 Dense 히든 레이어
        for unit in units :
            self.smodel.add(tf.keras.layers.Dense(unit,  activation='relu'))
        self.smodel.add(tf.keras.layers.Dense(units=1))
        
        #  Reshape 층 추가 : 단일 예측을 위해 (outputs) => (1, outputs) 1D를 2D로 변환해줌
        self.smodel.add(tf.keras.layers.Reshape([1, -1])) 
        self.name = name

    def call(self, inputs):
        return self.smodel(inputs)

In [ ]:
conv_fc = ConvFullyConnectedModel()
conv_fc_eval = conv_fc.execute(conv_window)
conv_fc_eval

In [ ]:
try :
    conv_fc.execute(wide_window)
except Exception as e :
    print(e)
# 이 에러는 입력 데이터의 형태가 모델이 기대하는 형태와 맞지 않아서 발생한다.
# 모델의 Dense 레이어는 입력 데이터의 마지막 축(axis -1)이 57(19 x 3)이길 기대하지만,
# wide_window는 (None, 24, 19) 형태로 입력 되고 있기 때문

CNN 모델

In [26]:
class ConvModel(tf.keras.Model, WindowPluginMixin):
    def __init__(self, units=[32, 32, 32], name='Conv-Model'):
        super().__init__()
        # 모델링
        self.smodel = tf.keras.Sequential()
        # Conv1D 레이어 추가 (여러 타입 스텝 처리 가능)
        self.smodel.add(
            tf.keras.layers.Conv1D(
                filters=units[0],
                kernel_size=(CONV_WIDTH,), # CONV_WIDTH = 3 으로 정의했었다.
                activation='relu'
            )
        )
        
        # 기존 Dense 히든 레이어
        for unit in units[1:] :
            self.smodel.add(tf.keras.layers.Dense(unit,  activation='relu'))
        self.smodel.add(tf.keras.layers.Dense(units=1))
        
        # 컨볼루션이 출력에서 시간 축을 유지하므로 Reshape 레이어가 필요 없음
        # self.smodel.add(tf.keras.layers.Reshape([1, -1])) 
        self.name = name

    def call(self, inputs):
        return self.smodel(inputs)

In [ ]:
conv = ConvModel()
conv_eval = conv.execute(conv_window)
conv_eval

In [ ]:
single_eval_df = pd.concat([baseline_single_eval, fc_eval, conv_fc_eval, conv_eval])
print(single_eval_df)
single_eval_df.iplot(kind='bar', title="싱글스텝 예측 모델 성능 비교")

___
### Shift1 멀티스텝 예측 모델
___

베이스라인 모델

In [ ]:
class Baseline(tf.keras.Model, WindowPluginMixin):
  def __init__(self, label_index=None, name='Baseline'):
    super().__init__()
    self.label_index = label_index
    self.name = name

  def call(self, inputs):
    if self.label_index is None:
      return inputs
    result = inputs[:, :, self.label_index] # 레이블 값을 리턴
    return result[:, :, tf.newaxis]

In [ ]:
base_line_wide_window = WindowGenerator(name="24Step 24Label 1Shift 윈도우", input_width=24, label_width=24, shift=1, label_columns=None)
base_line_wide_window

In [ ]:
baseline = Baseline()
baseline_single_eval = baseline.execute(base_line_wide_window)
baseline_single_eval

선형 모델

In [29]:
class LinearModel(tf.keras.Model, WindowPluginMixin):
    def __init__(self, name='LinearModel'):
        super().__init__()
        self.dense = tf.keras.layers.Dense(units=1)
        self.name = name

    def call(self, inputs):
        return self.dense(inputs)

In [ ]:
linear = LinearModel()
linear_wide_eval = linear.execute(wide_window)
linear_wide_eval

* Check Feature Importance

In [ ]:
# 선형 모델은 Feature Importance 확인이 가능하다. 모든 모델링의 시작전에 선형모델을 우선 진행하는 가장 이유중 하나는 설명력이다.
weights = linear.layers[0].kernel.numpy().reshape(-1)
xticks_labels = list(df.columns)

fig = px.bar(x=xticks_labels, y=weights, labels={'x': '특성 변수', 'y': '가중치'}, title='Feature Importance')
fig.show()

In [ ]:
mul_conv_wide_eval = mul_conv.execute(wide_conv_window)
mul_conv_wide_eval

LSTM 모델

In [ ]:
class LSTMModel(tf.keras.Model, WindowPluginMixin):
    def __init__(self, return_sequences=True, name='LSTM-Model'):
        super().__init__()
        # 모델링
        self.smodel = tf.keras.Sequential()
        # LSTM 레이어 추가 (여러 타입 스텝 처리 가능) [batch, time, features] => [batch, time, lstm_units]
        self.smodel.add(
            tf.keras.layers.LSTM(32, return_sequences=return_sequences), # return_sequences
        )
        # 출력층
        self.smodel.add(tf.keras.layers.Dense(units=1)) # 마지막 차원(즉, label의 크기)을 1로 설정
        # sequence 모델은 출력층에서 시스를 유지하고, single 모델일 경우는, 시퀀스 층이 없기 때문에, 이를 동일하게 맞춰준다. (batch, time, 1)  => (batch, time, 1) => or (batch, 1) => (None, time, 1)
        self.smodel.add(tf.keras.layers.Reshape((-1, 1)))
        
        self.name = name

    def call(self, inputs):
        return self.smodel(inputs)

* 멀티스텝 예측 LSTM모델

In [ ]:
# 입력t + 1 타임스텝부터 label크기만큼의 모든 타임스텝을 예측하는 모델 (초기 시퀀스는 성능을 기대하기 어렵다.)
lstm = LSTMModel(name='LSTM-Model-seq')
lstm_eval = lstm.execute(wide_window)
lstm_eval

 * 단일스텝 예측 LSTM 모델

In [ ]:
# 입력 타임스텝 이후 단일 타임스텝만 예측하는 모델
lstm_none_seq = LSTMModel(return_sequences=False, name='LSTM-Model-single')
lstm_none_seq_evel = lstm_none_seq.execute(wide_to_single_window)
lstm_none_seq_evel

In [ ]:
mul_shft1_eval_df = pd.concat([linear_wide_eval, mul_conv_wide_eval, lstm_eval])
mul_shft1_eval_df.iplot(kind="bar", title="Shift1 멀티스텝 예측 모델 성능 비교")

In [ ]:
single_eval_df = pd.concat([baseline_single_eval, fc_wide_eval, mul_fc_conv_eval, mul_conv_eval, lstm_none_seq_evel])
print(single_eval_df)
single_eval_df.iplot(kind='bar', title="싱글스텝 예측 모델 성능 비교(LSTM 포함)")

### #퀴즈. 24개 타입스텝 입력으로 그 다음 12개 타임스텝을 예측하기 위한 윈도우 제네레이터와 모델을 설계하고, 결과를 확인해보세요.
* 조건1 : LSTM 계열을 사용해보세요.

In [ ]:
# 윈도우 제네레이터 객체 생성

In [ ]:
# 모델 클래스 생성

In [ ]:
# 결과 확인

___
### ❗️❗️ 매운맛 모델 (Feedback 모델)
* 이전에 배운 모델을은 모두 싱글샷 모델입니다. 한번에 모든 타임스텝의 값을 예측하기 때문입니다.
* feedback 모델은 타임스텝을 하나씩 예측하면서 그 내용을 피드백받아 순차적으로 예측합니다.
___

멀티스텝 베이스라인 모델 (Reapater)
* 단순히 입력타입스텝 값을 반복합니다.

In [ ]:
OUT_STEPS = 24
baseline_muiti_window = WindowGenerator(name="24Step 24Label 24Step 윈도우 ", input_width=OUT_STEPS, label_width=OUT_STEPS, shift=OUT_STEPS, label_columns=None)
baseline_muiti_window

In [ ]:
class MultiStepBaselineModel(tf.keras.Model, WindowPluginMixin):
    def __init__(self):
        super().__init__()
        self.name = 'Multi-BaselineModel'
    def call(self, inputs):
        return inputs
    
mul_baseline = MultiStepBaselineModel()
mul_baseline_eval = mul_baseline.execute(baseline_muiti_window, training=False)
mul_baseline_eval

Feedback 모델

In [32]:
class FeedBackModel(tf.keras.Model, WindowPluginMixin):
    def __init__(self, units, out_steps=OUT_STEPS, name='LSTM-FeedBack-Model'):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
         # 레이어 준비 (순환하면서 훈련하고 각 타임스텝의 훈련 상태값을 그다음 레이어에 활용해야하기 때문에 모델구성을 모델 훈련/예측 단계에서 순차진행한다.)
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.fc = tf.keras.layers.Dense(19) # 입력데이터가 19개의 특성을 갖고 있으므로 배치수 x 특성수 로 변환하는 FC19 층을 준비한다.
        self.output_layer = tf.keras.layers.Dense(1) # 멀티레이블 에측이 아니므로 FC1 층을 하나더 준비한다.

        self.name = name

    # 모델링 및 예측로직 구성
    def call(self, inputs, training=None):
        # 예측값을 저장할 리스트 초기화
        predictions = []
        # 초기 예측값과 상태를 얻음
        x, *states = self.lstm_rnn(inputs)
        x = self.fc(x)
        prediction = self.output_layer(x)
        predictions.append(prediction)

        # out_steps - 1 만큼 반복하여 예측값 생성
        for _ in range(1, self.out_steps):
            # LSTM 셀을 통해 새로운 상태와 예측값을 얻음
            x, states = self.lstm_cell(x, states=states, training=training) # training 매개변수는 훈련시에만 True가 되고, 테스트시에는 False가 된다.
            # 완전 연결 층을 통해 최종 예측값 생성
            x = self.fc(x)
            prediction = self.output_layer(x)
            # 예측값을 리스트에 추가
            predictions.append(prediction)
            
        # 예측값 리스트를 텐서로 변환
        predictions = tf.stack(predictions)
        # 예측값 텐서의 차원 순서를 변경
        predictions = tf.transpose(predictions, [1, 0, 2])
        # 최종 예측값 반환
        return predictions


In [ ]:
feedback = FeedBackModel(units=32, out_steps=OUT_STEPS)
feedback_eval = feedback.execute(multi_window)
feedback_eval

싱글샷 선형모델 (성능 비교를 위해)

In [ ]:
class MultiStepLinearModel(tf.keras.Model, WindowPluginMixin):
    def __init__(self, name='SingleShot-LinearModel'):
        super().__init__()
        self.smodel = tf.keras.Sequential()
        self.smodel.add(tf.keras.layers.Lambda(lambda x: x[:, -1:, :], name='lambda_layer'))
        self.smodel.add(tf.keras.layers.Dense(OUT_STEPS, kernel_initializer=tf.initializers.zeros(), name='dense_layer'))
        self.smodel.add(tf.keras.layers.Reshape([OUT_STEPS, 1], name='reshape_layer'))
        self.name = name
        
    def call(self, inputs) :
        return self.smodel(inputs)

In [ ]:
mul_linear_model = MultiStepLinearModel()
mul_linear_model_eval = mul_linear_model.execute(multi_window)
mul_linear_model_eval

* 멀티 스텝 성능 비교

In [ ]:
mul_eval_df = pd.concat([mul_baseline_eval, mul_linear_model_eval, feedback_eval])
print(mul_eval_df)
mul_eval_df.iplot(kind='bar')